# 1.PromptTemplate

## 1.1、使用说明

 - PromptTemplate类,用于快速构建 **包含变量** 的提示词模板,并通过 **传入不同的参数值** 生成自定义的提示词
 - 主要参数介绍
  - template: 定义提示词模板的字符串,包含文本和变量占位符
  - input_variables: 变量列表,指定模板中使用的变量名称,在调用模板时被替换
  - partial_variables: 变量字典,用于定义模板中一些固定的变量名,这些变量不需要在每次调用时被替换
 - 函数介绍:
  - format():
    - 给input_variables变量赋值,并返回模板格式化后的字符串
    - 使用format()格式化时就一定要赋值,不然会报错

## 1.2、PromptTemplate的实例化

- 1）使用PromptTemplate构造方法

In [85]:
from itertools import product

from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate(
    template="你叫{name},是一个{personality}的{role}",
    input_variables={"name", "personality", "role"}
)

prompt = prompt_template.format(name = "小智", personality = "博学多才,耐心细致", role = "人工智能领域的专家")

print(prompt)


你叫小智,是一个博学多才,耐心细致的人工智能领域的专家


- 2）调用from_template()

In [86]:
prompt_template2 = PromptTemplate.from_template("你叫{name},是一个{personality}的{role}")

prompt2 = prompt_template2.format(name="阿智", personality="追求极致", role="AI应用开发工程师")
print(prompt2)

你叫阿智,是一个追求极致的AI应用开发工程师


模板支持任意数量的变量，包括不含变量：

In [87]:
# 定义提示词模版对象
text = """
Tell me a joke
"""
prompt_template = PromptTemplate.from_template(text)
# 默认使用f-string进行格式化（返回格式好的字符串）
prompt = prompt_template.format()
print(prompt)


Tell me a joke



## 1.3、部分提示词模板

- 在生成Prompt前,就提前初始化 部分提示词,实际进一步导入模板时 只导入除已初始化的变量即可

示例1: 实例化Prompt时使用partial_variables变量

In [25]:
partial_prompt1 = PromptTemplate.from_template(
    template="请介绍下{product}各方面的特点,比如: {aspect1},{aspect2},{aspect3}",
    partial_variables={"product": "小米手机", "aspect1": "电池续航","aspect2":"像素","aspect3": "屏幕大小"},   # partial_variables: dict[str, Any] | None = None,
)
prompt = partial_prompt1.format()
print(f"partial_prompt: {prompt}")

partial_prompt: 请介绍下小米手机各方面的特点,比如: 电池续航,像素,屏幕大小


实例2: 使用partial()方法创建部分提示模板

In [88]:
partial_prompt2 = PromptTemplate(
    template="请介绍下{product}各方面的特点,比如: {aspect1},{aspect2},{aspect3}",
    input_variables=["product","aspect1","aspect2","aspect3"]
).partial(
    product="小米手机",
    aspect1="电池续航",
    aspect2="像素",
    aspect3="屏幕大小")

prompt2 = partial_prompt2.format()
print(f"partial_prompt2:{prompt2}")

partial_prompt2:请介绍下小米手机各方面的特点,比如: 电池续航,像素,屏幕大小


In [89]:
partial_prompt3 = PromptTemplate(
    template="请介绍下{product}各方面的特点,比如: {aspect1},{aspect2},{aspect3}",
    input_variables=["product","aspect1","aspect2","aspect3"]
).partial(
    aspect1="电池续航",
    aspect2="像素")
# 设置其他属性
prompt3 = partial_prompt3.format(product="华为手机",aspect3="屏幕大小")
print(f"partial_prompt3:{prompt3}")

partial_prompt3:请介绍下华为手机各方面的特点,比如: 电池续航,像素,屏幕大小


## 1.4、format()和invoke()

- 调用format():  入参类型,给变量赋值的PromptTemplate; 返回值类型,格式化后(给变量赋值)的字符串

In [28]:
prompt_template = PromptTemplate.from_template("你叫{name},是一个{personality}的{role}")

format_prompt = prompt_template.format(name="阿智", personality="追求极致", role="AI应用开发工程师")
print(format_prompt)
print(type(format_prompt))

你叫阿智,是一个追求极致的AI应用开发工程师
<class 'str'>


- 调用invoke(): 入参类型为dict字典类型; 返回值类型为StringPromptValue

In [29]:
prompt_template = PromptTemplate.from_template("你叫{name},是一个{personality}的{role}")

invoke_prompt = prompt_template.invoke(input={"name":"小小智", "personality":"追求极致", "role":"AI应用开发工程师"})
print(invoke_prompt)
print(type(invoke_prompt))

text='你叫小小智,是一个追求极致的AI应用开发工程师'
<class 'langchain_core.prompt_values.StringPromptValue'>


In [ ]:
## 1.4、调用大模型

In [30]:
from langchain_openai import ChatOpenAI
import os
import dotenv

dotenv.load_dotenv()

chatModel = ChatOpenAI(
    model = "deepseek-chat",
    base_url = os.environ["BASE_URL"],
    api_key = os.environ["DEEPSEEK_API_KEY"]
)

prompt_template = PromptTemplate.from_template("请在{aspect1}、{aspect2}等维度来简单介绍{product},要求每个维度用一两句话描述")

llm_prompt = prompt_template.format(aspect1="生态",aspect2="性能",product="LlamaIndex")    # 类型为<class 'str'>
# llm_prompt = prompt_template.invoke(input={"aspect1":"生态","aspect2":"性能","product":"LlamaIndex"})  # 类型为<class 'langchain_core.prompt_values.StringPromptValue'>

print(type(llm_prompt))

chatModel.invoke(llm_prompt)

<class 'str'>


AIMessage(content='好的，从生态和性能维度简单介绍LlamaIndex：\n\n**生态维度**\nLlamaIndex拥有强大的数据连接能力，能无缝集成各类数据源、LLM和向量数据库，构建了一个繁荣的插件工具链生态，极大简化了数据接入与应用分发的流程。\n\n**性能维度**\n它通过高效的索引结构、检索策略与查询引擎，在保证高精度检索的同时，显著提升了RAG应用的整体响应速度与吞吐量。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 24, 'total_tokens': 117, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 24}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'd3050e2d-4724-4a93-9cef-d04cb7d96b2d', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--fdd209c8-1a9e-4784-9166-eaa889a57b7f-0', usage_metadata={'input_tokens': 24, 'output_tokens': 93, 'total_tokens': 117, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

# 2、ChatPromptTemplate

## 2.1、使用说明
- ChatPromptTemplate是创建**聊天消息列表**的提示词模板,比PromptTemplate更适合处理多角色、多轮次的对话场景
- 特点:
  - 支持System、Human、AI等不同角色的消息模板,如果不指定角色,则默认是Human
  - 维护对话历史
- 参数类型:
  - 列表参数为tuple类型,tuple每个元组对应一条结构化的对话消息模板,元组的两个元素 (role,content) 分别定义了消息的角色和消息的内容模板
    - 格式为 (role:str|type,content:str|list[dict]|list[object])
    - role可以是字符串(如:"System"、"human"、"ai"),也可以是消息类 类型(如:SystemMessage、HumanMessage、AIMessage)
    - content可以是字符串、字典列表或对象列表

## 2.2、ChatPromptTemplate的实例化

- 1）使用构造方法

In [35]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate(
    [
        ("system", "你叫{name},是一名资深的{vocation}"),
        ("human", "你会哪些技能?"),
        ("ai", "我熟悉{skill}等多种主流AI技术"),
        ("human", "请用两三句话介绍下{keyword}")
    ]
)
# chat_prompt = chat_prompt.invoke(input={"name":"xuzhixing","vocation":"AI应用开发工程师","skill":"RAG、Agent、大模型微调","keyword":"RAG和Agent"})
chat_prompt = chat_prompt.format(name="xuzhixing",vocation="AI应用开发工程师",skill="RAG、Agent、大模型微调",keyword="RAG和Agent")

print(chat_prompt)
print(type(chat_prompt))

System: 你叫xuzhixing,是一名资深的AI应用开发工程师
Human: 你会哪些技能?
AI: 我熟悉RAG、Agent、大模型微调等多种主流AI技术
Human: 请用两三句话介绍下RAG和Agent
<class 'str'>


- 2）调用from_messages(): 可指定对话的角色和内容,角色有system、human、ai,如果不指定角色,则默认是human【用户】

In [37]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你叫{name},是一名资深的{vocation}"),
        ("human", "你会哪些技能?"),
        ("ai", "我熟悉{skill}等多种主流AI技术"),
        ("human", "请用两三句话介绍下{keyword}")
    ]
)
# chat_prompt = chat_prompt.invoke(input={"name":"xuzhixing","vocation":"AI应用开发工程师","skill":"RAG、Agent、大模型微调","keyword":"RAG和Agent"})
chat_prompt = chat_prompt.format(name="xuzhixing",vocation="AI应用开发工程师",skill="RAG、Agent、大模型微调",keyword="RAG和Agent")

print(chat_prompt)
print(type(chat_prompt))

messages=[SystemMessage(content='你叫xuzhixing,是一名资深的AI应用开发工程师', additional_kwargs={}, response_metadata={}), HumanMessage(content='你会哪些技能?', additional_kwargs={}, response_metadata={}), AIMessage(content='我熟悉RAG、Agent、大模型微调等多种主流AI技术', additional_kwargs={}, response_metadata={}), HumanMessage(content='请用两三句话介绍下RAG和Agent', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.prompt_values.ChatPromptValue'>


- 模板调用的函数:  invoke()、format()、format_messages()、format_prompt()
- 给占位符赋值,针对ChatPromptTemplate,推荐使用from_messages()方法,返回消息列表
- 创建ChatPromptTemplate时,不管是使用构造方法,还是使用from_messages(),参数是列表类型,列表中的元素可以是字符串、字典、字符串构成的元组、消息类型、提示词模板类型、消息提示词模板类型等

# 3、少量样本示例的提示词模板

## 3.1、使用说明

- FewShotPromptTemplate/FewShotChatMessagePromptTemplate:在构建Prompt时,可通过构建一个**少量示例列表**来格式化Prompt,简单且强大、稳定且可靠的指导回答生成【尽可能地减少幻觉】, 在某些情况下可以显著提高大模型的性能
    - FewShotPromptTemplate:
        - 输出内容为字符串,example_prompt属性为 PromptTemplate,不支持历史对话
        - 适用场景: 文本补全模型、单次输入输出类任务、文本分类、摘要、代码生成、翻译等
    - FewShotChatMessagePromptTemplate:用于聊天对话模式
        - 输出内容为消息列表,example_prompt属性为 ChatPromptTemplate,支持历史对话
        - 适用场景: 聊天对话模型、多轮对话交互、智能客服、情感分析、代码助手等
- 少量示例提示模板可以由**一组示例** 或 一个负责从定义的集合中选择**一部分示例**的示例选择器构成

## 3.2、FewShotPromptTemplate

In [64]:
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate

# 1.创建示例集合
few_shot = [
    {"question": "请介绍下深圳", "answer": "深圳是广东省的地级市,其车牌代码为粤B"},
    {"question": "请介绍下武汉", "answer": "武汉是湖北省的省会、地级市,其车牌代码为鄂A"}
]

# 2.创建PromptTemplate实例
shot_prompt = PromptTemplate.from_template(
    template="question: {question}\nanswer: {answer}"
)

# 3.创建FewShotPromptTemplate实例
fewshot_prompt = FewShotPromptTemplate(
    examples=few_shot,
    example_prompt=shot_prompt,
    suffix="question: {question}\nanswer:", # 要放在示例后面的提示模板字符串
    input_variables=["question"]
)

# 4.调用大模型
prompt1 = fewshot_prompt.format(question="请介绍下长沙")
response1 = chatModel.invoke(prompt1)
print(response1)

prompt2 = fewshot_prompt.format(question="请介绍下宁波")
response2 = chatModel.invoke(prompt2)
print(response2)

print(response1.content)
print(response2.content)

content='长沙是湖南省的省会、地级市，其车牌代码为湘A' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 67, 'total_tokens': 83, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 64}, 'prompt_cache_hit_tokens': 64, 'prompt_cache_miss_tokens': 3}, 'model_provider': 'openai', 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_ffc7281d48_prod0820_fp8_kvcache', 'id': 'e2856a3f-a3c2-4cea-a033-ed7fdd32758a', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--172f10c8-e093-4ba4-ad82-5440b04ccd17-0' usage_metadata={'input_tokens': 67, 'output_tokens': 16, 'total_tokens': 83, 'input_token_details': {'cache_read': 64}, 'output_token_details': {}}
content='宁波是浙江省的副省级市、计划单列市，其车牌代码为浙B。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 67, 'total_tokens': 87, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': No

## 3.3、FewShotChatMessagePromptTemplate

- FewShotChatMessagePromptTemplate是专门为**聊天对话场景**设计的少样本(few-shot)提示模板,继承自FewShotPromptTemplate,并优化了聊天消息的格式
- 特点:
  - 自动将示例格式化为聊天消息(HumanMessage/AIMessage等)
  - 输出结构化的聊天消息(List[BaseMessage])
  - 保留对话的轮次结构

- 代码示例:

In [74]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

# 1. 定义示例（每个示例包含输入和输出）
few_shot = [
    {"input": "这部电影太棒了！", "output": "正向评价"},
    {"input": "今天天气真糟糕。", "output": "负面评价"},
    {"input": "这个产品还可以。", "output": "中性评价"}
]

# 2. 创建示例格式化模板（必须使用 ChatPromptTemplate）
shot_prompt = ChatPromptTemplate.from_messages([
    ("human", "{input}"),
    ("ai", "{output}")
])

# 3. 创建 FewShotChatMessagePromptTemplate
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=few_shot,
    example_prompt=shot_prompt,
    input_variables=["input"]
)

# 4. 嵌入到 ChatPromptTemplate 中使用
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是评价分析助手。根据示例判断评价好坏"),
    few_shot_prompt,  # 自动插入 few-shot 示例
    ("human", "{input}")
])

messages = final_prompt.format_messages(input="今天这顿饭真好吃！")
# 5. 调用对话模型
response = chatModel.invoke(messages)
print(f"输入: 今天这顿饭真好吃！")
print(f"输出: {response.content}")


messages = final_prompt.format_messages(input="那个车还说得过去吧！")
response = chatModel.invoke(messages)
print(f"输入: 那个车还说得过去吧！")
print(f"输出: {response.content}")


输入: 今天这顿饭真好吃！
输出: 正向评价
输入: 那个车还说得过去吧！
输出: 中性评价


## 3.4、Example selectors(示例选择器)

- FewShotPromptTemplate和FewShotChatMessagePromptTemplate的特点: 无论输入什么问题,都会包含全部示例
- 在实际项目中,我们通常会根据当前输入, 使用示例选择器, 从大量候选示例中选取最相关的示例子集
    - 使用的好处: 避免盲目传递所有示例, 降低噪音, 减少token消耗, 提升输出效果
    - 示例选择策略: 语义相似选择、长度选择、最大边际相关示例选择等
        - 语义相似选择: 通过余弦相似度、欧式距离等度量方式评估语义相关性, 选择和输入问题最相关的k个示例
        - 长度选择: 根据输入文本的长度, 从候选示例中筛选出长度最匹配的示例, 增强模型对文本结构的理解, 比语义相似度计算更轻量, 适合对响应速度要求高的场景
        - 优先选择 和 输入问题语义相似的示例, 同时通过惩罚机制来避免返回同质化的内容

- 在如下的代码示例可分为 检索前处理和检索后处理
    - 1）examples示例集中的问答示例 经由嵌入模型 转换为向量后,存储在Chroma库
    - 2）对给定的问题通过嵌入模型生成向量,查询Chroma库 逐个比对&筛选, 得到最相关的2个示例

In [83]:
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
import os
import dotenv

dotenv.load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['OPENAI_BASE_URL'] = os.getenv("OPENAI_BASE_URL")

# 1. 初始化嵌入模型
embeddings_model = OpenAIEmbeddings(
    model="text-embedding-ada-002"
)

# 2. 定义示例组（包含多个不同主题的问答示例）
examples = [
    {"question": "Python是什么？", "answer": "Python是一种高级编程语言，以其简洁的语法和强大的功能而闻名。"},
    {"question": "什么是机器学习？", "answer": "机器学习是人工智能的一个分支，让计算机通过数据学习而无需明确编程。"},
    {"question": "如何学习编程？", "answer": "学习编程需要多实践、多写代码，可以从基础语法开始，逐步学习算法和项目开发。"},
    {"question": "什么是深度学习？", "answer": "深度学习是机器学习的一个子领域，使用神经网络来模拟人脑的学习过程。"},
    {"question": "LangChain是什么？", "answer": "LangChain是一个用于构建基于大语言模型应用的框架，提供了丰富的工具和组件。"},
    {"question": "如何安装Python？", "answer": "可以从Python官网下载安装包，或使用Anaconda等发行版，安装后配置环境变量即可。"},
    {"question": "什么是向量数据库？", "answer": "向量数据库是专门用于存储和检索高维向量数据的数据库，常用于相似度搜索。"},
    {"question": "如何学习人工智能？", "answer": "学习AI需要掌握数学基础、编程技能，然后学习机器学习、深度学习等专业知识。"},
]

# 3. 定义示例选择器
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # 可供选择的示例列表
    examples=examples,
    # 指定生成向量的嵌入模型，用于衡量语义相似性
    embeddings=embeddings_model,
    # 存储嵌入并进行相似性搜索的 VectorStore 类
    vectorstore_cls=Chroma,
    # 指定生成的示例数量（选择最相似的k个示例）
    k=2,
)

# 4. 测试：不同问题会选择不同的相关示例（核心特点展示）
print("=" * 70)
print("示例选择器核心特点:根据输入动态选择最相关的示例")
print("=" * 70)

test_questions = [
    "我想了解Python编程语言",        # 应该选择"Python是什么？"相关示例
    "机器学习的基本概念是什么？",      # 应该选择"什么是机器学习？"相关示例
    "如何开始学习编程？",              # 应该选择"如何学习编程？"相关示例
    "什么是神经网络？",                # 应该选择"什么是深度学习？"相关示例
]

for question in test_questions:
    print(f"\n问题: {question}")
    print("-" * 70)
    
    # 选择与输入最相似的示例
    selected_examples = example_selector.select_examples({"question": question})
    
    print(f"选择的示例数量: {len(selected_examples)}")
    print("选择的示例:")
    
    for i, example in enumerate(selected_examples, 1):
        print(f"\n  示例 {i}:")
        for k, v in example.items():
            print(f"    {k}: {v}")
    
    print("-" * 70)

示例选择器核心特点:根据输入动态选择最相关的示例

问题: 我想了解Python编程语言
----------------------------------------------------------------------
选择的示例数量: 2
选择的示例:

  示例 1:
    question: Python是什么？
    answer: Python是一种高级编程语言，以其简洁的语法和强大的功能而闻名。

  示例 2:
    question: 如何安装Python？
    answer: 可以从Python官网下载安装包，或使用Anaconda等发行版，安装后配置环境变量即可。
----------------------------------------------------------------------

问题: 机器学习的基本概念是什么？
----------------------------------------------------------------------
选择的示例数量: 2
选择的示例:

  示例 1:
    answer: 机器学习是人工智能的一个分支，让计算机通过数据学习而无需明确编程。
    question: 什么是机器学习？

  示例 2:
    question: 什么是深度学习？
    answer: 深度学习是机器学习的一个子领域，使用神经网络来模拟人脑的学习过程。
----------------------------------------------------------------------

问题: 如何开始学习编程？
----------------------------------------------------------------------
选择的示例数量: 2
选择的示例:

  示例 1:
    question: 如何学习编程？
    answer: 学习编程需要多实践、多写代码，可以从基础语法开始，逐步学习算法和项目开发。

  示例 2:
    answer: 学习AI需要掌握数学基础、编程技能，然后学习机器学习、深度学习等专业知识。
    question: 如何学习人工智能？
-------------------